In [1]:
import pandas as pd
import numpy as np
import time
from src.peblMl import *

In [6]:
%%time
data_start=pd.read_pickle('data/wikiNews.pkl')
data_train=data_start[:3000]
data_test=data_start[3000:]
#data_test=data_test[~data_test['topic'].isin(['other'])]
data_train=data_train[~data_train['topic'].isin(['other'])]
print (len(data_test), len(data_train))

7748 1160
Wall time: 131 ms


In [7]:
d_t=data_test[~data_test['topic'].isin(['other'])]
print (len(d_t))

395


In [8]:
data_train.head()

,Noun,Text,category,topic,date_news
7376,мефодий год жизнь поэт член союз писатель росс...,владимир мефодьевич сегодня из пришло известие...,барнаул алтайский_край некрологи писатели росс...,"россия, некрологи",2005-02-01
1349,россия премьерминистр украина разъяснение вним...,россия заявлениями премьерминистра украины хоч...,россия украина политика вто юрий_ехануров поли...,"россия, украина, политика, европа",2005-11-01
1520,ангел меркель германия германия женщина правит...,ангела меркель канцлер германии германия она п...,ангела_меркель христианско_демократический_сою...,политика,2005-11-01
1527,атака ноябрь страница атака домен машина гугл ...,атака гугл данным компании ноября предпринята ...,google швейцария европа интернет хакерские_ата...,европа,2005-11-01
1531,ограничение технология доставка мкс состояние ...,ограничения закупку космических технологий кос...,космонавтика мкс политика_сша джордж_уокер_буш...,политика,2005-11-01


In [9]:
%%time
modelML=peblMl()
modelML.fit(data_train['Noun'].tolist(), data_train['topic'].tolist())

Wall time: 305 ms


In [28]:
modelML.predict(data_train['Noun'].tolist()[11], 'newTopic')[:10]

политика      0.648028
россия        0.149889
европа        0.114426
футбол        0.020103
спорт         0.018300
некрологи     0.016339
франция       0.010750
астрономия    0.009889
украина       0.008861
праздники     0.003416
dtype: float64

In [19]:
%%time
topicValueResult=pd.DataFrame(columns=['topicName', 'probValue', 'realTopic'])
goodT=0
wrongT=0
i=0
for testRow in data_test['Noun'].tolist():
    result=modelML.predict(testRow, 'other')[:1]
    topicValueResult=topicValueResult.append({'topicName': result.index[0], 'probValue': result.values[0], 
                                             'realTopic': data_test['topic'][i]}, ignore_index=True)
    if result.index[0]!='other' and data_test['topic'][i]!='other':
        print (result.index[0], result.values[0], data_test['topic'][i])
        goodT+=1
    if data_test['topic'][i]!='other' and result.index[0]=='other':
        wrongT+=1
    i+=1
    #if i%10==0:
    #    print (result.index[0], result.values[0])


политика 0.4158081920300238 россия, политика
политика 0.3707877152064413 политика
политика 0.38194880846932877 политика
политика 0.5380239545690155 политика
европа 0.21521399313500028 футбол
политика 0.2694180165477 россия
политика 0.42682188639608076 политика
политика 0.3658509853728339 политика
политика 0.2967250055558436 россия, европа
политика 0.470574001955913 политика
политика 0.3488030564275795 россия
политика 0.4476135686093295 политика
россия 0.20841591198250148 украина
политика 0.5806795805586235 политика
политика 0.5748740911041084 политика
политика 0.31545258914166824 политика, некрологи
россия 0.23819535347784637 россия
политика 0.2895430823121268 россия, некрологи
россия 0.22077541612794604 россия
политика 0.5395104662861641 политика
политика 0.3911336609731711 политика
политика 0.31768257697106733 россия
политика 0.29692067112321713 футбол, спорт
политика 0.40280873019805813 россия, политика
политика 0.4781343157392098 политика
политика 0.39079886655972895 россия, полити

политика 0.3908063179934226 политика
россия 0.28123985098940835 спорт
политика 0.38890348106522354 политика
астрономия 0.3935033028546996 астрономия
политика 0.4152465212787765 политика
политика 0.2620791117667013 украина, европа, спорт
россия 0.24342349310253458 спорт
политика 0.37557589887756937 политика
политика 0.2760296240542485 россия
политика 0.4662919349739928 политика
россия 0.20278020403326438 украина, футбол, спорт
политика 0.3266833685121146 праздники
политика 0.3283108844048157 спорт
политика 0.2513265160682408 спорт
политика 0.2811922944551254 европа
политика 0.25866458650083696 спорт
политика 0.2355889041749726 астрономия
политика 0.3388663396448251 россия
политика 0.35591194371674856 спорт
россия 0.19742674782299935 европа, спорт
политика 0.2513743251884532 украина
политика 0.21086066955049584 астрономия
политика 0.4152786516690709 европа
россия 0.23470074570000768 футбол
политика 0.27260408342333164 спорт
политика 0.2095520362804868 футбол
россия 0.2303577728430021 фут

In [21]:
print (wrongT, goodT)

120 275


In [16]:
topicValueResult.head() 

,topicName,probValue
0,политика,0.415808
1,other,0.357354
2,политика,0.303558
3,other,0.421407
4,политика,0.372710


In [18]:
topicValueResult['topicName'].value_counts()

other         3738
политика      3472
россия         392
европа          91
астрономия      26
спорт           21
футбол           5
некрологи        2
франция          1
Name: topicName, dtype: int64

In [20]:
topicValueResult['realTopic'].value_counts()

other                                        7353
политика                                       79
россия                                         78
спорт                                          40
европа                                         26
футбол                                         24
россия, политика                               22
некрологи                                      11
праздники                                      11
россия, европа                                 10
украина                                         9
футбол, спорт                                   9
россия, спорт                                   6
астрономия                                      6
россия, политика, европа                        6
европа, спорт                                   5
россия, некрологи                               5
франция                                         5
россия, футбол, спорт                           4
футбол, европа, спорт                           4


In [34]:
topicValueResult.groupby(['topicName']).sum()

,probValue
topicName,
newTopic,3.414692
европа,0.505284
политика,12.797851
россия,0.906963
